In [2]:
import cv2
import numpy as np
from scipy.stats import norm

def denoise(im,J,sigma,burn_in=25,it_cnt=500):
    # perform type conversions and ensure input \in {1,-1}
    noisy = im.astype(np.float32)
    noisy[noisy==255]=1
    noisy[noisy==0]=-1
    psi_t = {}
    psi_t['1'] = norm.pdf(noisy,1,sigma) # $\psi_t(+1)
    psi_t['-1'] = norm.pdf(noisy,-1,sigma) # $\psi_t(-1)
    avg = np.zeros(noisy.shape)
    
    sample = noisy.copy()# start from current observation
    for i in range(it_cnt):
        # direct implementation of equation 3 
        # note that we use convolution to perform summation for performance reasons
        k = np.array([[0,1,0],[1,0,1],[0,1,0]])
        nbr_sum = cv2.filter2D(sample,-1,k) # $\sum_{s\in nbr(t)}{x_s}$
        p1 = np.exp(J*nbr_sum) * psi_t['1']
        p0 = np.exp(-J*nbr_sum) * psi_t['-1']
        prob = p1/(p1+p0)
        
        sample_mask = np.less(np.random.random(noisy.shape),prob)
        sample[...]=-1
        sample[sample_mask]=1
        
        if i%100==0:
            print('it cnt %d\r'%i)
        if i>=burn_in:
            avg += sample
    avg /= (it_cnt-burn_in)
    
    avg[avg>0]=255
    avg[avg<=0]=0
    return avg.astype(np.uint8)
    



In [7]:
im = cv2.imread('3_noise.png')[:,:,0]
denoised=denoise(im,1,1,burn_in=5,it_cnt=15)
print('denoise done')
while True:
    cv2.imshow('noisy',im)
    cv2.imshow('denoised',denoised)
    if cv2.waitKey(1) &0xFF==27:
        break

it cnt 0
denoise done


In [29]:
sample_mask = np.less(np.random.random(noisy.shape),prob)
sample = np.zeros(noisy.shape)-1
sample[sample_mask]=1

In [31]:

cv2.imshow('im',sample*255)
cv2.waitKey(0)

1114089

array([[ 0.19947114,  0.12098536,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       [ 0.19947114,  0.19947114,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       [ 0.19947114,  0.19947114,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       ..., 
       [ 0.12098536,  0.12098536,  0.19947114, ...,  0.12098536,
         0.19947114,  0.12098536],
       [ 0.19947114,  0.12098536,  0.19947114, ...,  0.12098536,
         0.12098536,  0.19947114],
       [ 0.12098536,  0.12098536,  0.19947114, ...,  0.12098536,
         0.12098536,  0.19947114]])

In [24]:
k

array([[0, 1, 0],
       [1, 0, 1],
       [0, 1, 0]])